In [1]:
%pylab inline

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import math
import statsmodels
import statsmodels.api as sm
import scipy as sc
from sklearn import metrics
import itertools
import statsmodels.formula.api as smf
import sklearn.linear_model
import sklearn.preprocessing
from IPython.display import clear_output
import pickle
import statsmodels.api as sm

Populating the interactive namespace from numpy and matplotlib


c:\python27\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


Читаем подготовленные данные для сто двух ячеек регионов с каггл

In [2]:
#https://github.com/nlavri/mipt_ya_ml/blob/master/data/stats_kaggle_2015-03_2016-06.dat
data = pd.read_pickle('TaxiData/stats_kaggle_2015-03_2016-06.dat')
print data.shape
data.head()

(1194522, 2)


,region,count
tpep_pickup_datetime,,
2015-03-01,1075,47.0
2015-03-01,1076,103.0
2015-03-01,1077,51.0
2015-03-01,1125,99.0
2015-03-01,1126,287.0


Составим признаки для регресии по каждому региону. Данные каждого региона будем помещать в отдельный фрейм, а потом объединим в один общий, чтобы соответствовать постановке задачи и создать шесть моделей на все регионы сразу, а не на каждый регион по отдельности. 

После ряда пробных обучений оказалось, что в моем случае признаки основанные на дате, синусах\косинусах только мешают. При этом заметно помогает уменьшить ошибку увеличение признаков из состоящих из откликов T-k, поэтому менее полезные признаки ниже закоментированны. 

In [3]:
nModels = 6

#K = 60

start_time = pd.to_datetime('2015-09-01 00:00:00')
end_time = pd.to_datetime('2016-06-30 17:00:00')

frames = []

for reg in data.region.unique():
    clear_output(wait=True)
    print reg
    sample_data = data[data.region == reg]['count'].sort_index()
    
    #fix 2016-06-30 23:00:00
    sample_data[pd.to_datetime('2016-06-30 23:00:00')] = sample_data[pd.to_datetime('2016-06-30 22:00:00')]
    
    frame = pd.DataFrame()

    frame['data'] = sample_data[start_time:end_time]

    frame['region'] = [reg] * frame.shape[0]
    
    #for k in range(1, K + 1):
    #    frame.loc[:,'s'+ str(k)] = np.sin(np.arange(frame.shape[0]) * 2 * np.pi * k / 168)
    #    frame.loc[:,'c'+ str(k)] = np.cos(np.arange(frame.shape[0]) * 2 * np.pi * k / 168)

    #frame.loc[:, 'day'] = frame.index.map(lambda x: x.day)
    #frame.loc[:, 'month'] = frame.index.map(lambda x: x.month)
    #frame.loc[:, 'weekday'] = frame.index.map(lambda x: x.weekday())

    Khist = 48

    for k in range(1, Khist + 1):
        t_delta = datetime.timedelta(hours=k)
        frame.loc[:, 'data_history_' + str(k)] =  sample_data[start_time - t_delta:end_time - t_delta].values

    #Kday = 14
        
    #for k in range(1, Kday + 1):
    #    t_delta = datetime.timedelta(hours=24 * k)
    #    frame.loc[:, 'data_history_day_' + str(k)] =  sample_data[start_time - t_delta:end_time - t_delta].values
        
    for shift in range(1, nModels + 1):
        delta = datetime.timedelta(hours=shift)
        frame['target_' + str(shift)] = sample_data[start_time + delta:end_time + delta].values

    frames.append(frame)

1684


In [4]:
allData = pd.concat(frames)
allData = allData.sort_index()
print allData.shape

(743580, 56)


One Hot Encoding для категориальных признаков. В дальнейшем так же НЕ используются, т.к. не доказали свою полезность.

In [28]:
regionEncode = pd.get_dummies(allData['region'])

for col in regionEncode.columns:
    allData[col] = regionEncode[col]
    regionEncode = regionEncode.drop(col, axis=1)

In [29]:
weekDayEncode = pd.get_dummies(allData['weekday'])

for col in weekDayEncode.columns:
    allData['weekday_' + str(col)] = weekDayEncode[col]
    
dayEncode = pd.get_dummies(allData['day'])

for col in dayEncode.columns:
    allData['day_' + str(col)] = dayEncode[col]

Обучим модели на данных до конца апреля включительно (последнее Т это '2016-04-30 17:00:00' соответственно последний отклик для T+6 это отклик на '2016-04-30 23:00:00'. Используем регресиию Лассо с регуляризацией L1. В цикле обучение будем подбирать параметр альфа (диапазон выбран эмпирически на отдельных рядах). 
Для теста используем уже май.

In [28]:
train_from = pd.to_datetime('2016-02-01 00:00:00')
train_to = pd.to_datetime('2016-04-30 17:00:00')

x_names = []

x_names.append('region')
#for reg in allData['region'].unique():
#    x_names.append(reg)

#for col in weekDayEncode.columns:
#    x_names.append('weekday_' + str(col))

#for col in dayEncode.columns:
#    x_names.append('day_' + str(col))

#for k in range(1, K + 1):
#    x_names.append('s'+ str(k))
#    x_names.append('c'+ str(k))

x_names.append('data')    

for k in range(1, Khist + 1):
    x_names.append('data_history_' + str(k))
    
#for k in range(1, Kday + 1):
#    x_names.append('data_history_day_' + str(k))


test_from = pd.to_datetime('2016-04-30 23:00:00')
test_to = pd.to_datetime('2016-05-31 17:00:00')

alphas = np.arange(5, 100, 5)

topModels = {}
topAlphas = {}        
for shift in range(1, nModels + 1):
    bestModel = None
    bestAlpha = 0
    bestMae = sys.maxint

    for alpha in alphas:

        print 'shift', shift, 'alpha is ', alpha, 
        
        m = sklearn.linear_model.Lasso(alpha=alpha)

        trainF = allData[train_from : train_to]
        fitted = m.fit(trainF[x_names], trainF['target_' + str(shift)])

        mae = metrics.mean_absolute_error(allData[test_from : test_to]['target_' + str(shift)],
                                          fitted.predict(allData[test_from : test_to][x_names]))
        
        if mae < bestMae:
            bestMae = mae
            bestModel = fitted
            bestAlpha = alpha
        
        print 'MAE',  mae

    topModels[shift] = bestModel
    topAlphas[shift] = bestAlpha
    print 'shift', shift, 'best alpha is ', bestAlpha, 'MAE', bestMae


 shift 1 alpha is  5 MAE 21.122552556761615
shift 1 alpha is  10 MAE 21.10543962706564
shift 1 alpha is  15 MAE 21.094521128988323
shift 1 alpha is  20 MAE 21.08708584141159
shift 1 alpha is  25 MAE 21.08173492908044
shift 1 alpha is  30 MAE 21.080104082555252
shift 1 alpha is  35 MAE 21.080773273936845
shift 1 alpha is  40 MAE 21.084328949381007
shift 1 alpha is  45 MAE 21.089314678307822
shift 1 alpha is  50 MAE 21.09604076033201
shift 1 alpha is  55 MAE 21.105192436906275
shift 1 alpha is  60 MAE 21.116969163796185
shift 1 alpha is  65 MAE 21.13110411376902
shift 1 alpha is  70 MAE 21.149155111429494
shift 1 alpha is  75 MAE 21.167262320409833
shift 1 alpha is  80 MAE 21.187462423330693
shift 1 alpha is  85 MAE 21.208948338370114
shift 1 alpha is  90 MAE 21.23260094971326
shift 1 alpha is  95 MAE 21.258498732703305
shift 1 best alpha is  30 MAE 21.080104082555252
shift 2 alpha is  5 MAE 28.864900445905022
shift 2 alpha is  10 MAE 28.839860598610176
shift 2 alpha is  15 MAE 28.824322

In [29]:
topAlphas

{1: 30, 2: 45, 3: 50, 4: 65, 5: 95, 6: 85}

Уточним альфы для каждой из шести моделей

In [38]:
train_from = pd.to_datetime('2016-02-01 00:00:00')
train_to = pd.to_datetime('2016-04-30 17:00:00')

x_names = []

x_names.append('region')
#for reg in allData['region'].unique():
#    x_names.append(reg)

#for col in weekDayEncode.columns:
#    x_names.append('weekday_' + str(col))

#for col in dayEncode.columns:
#    x_names.append('day_' + str(col))

#for k in range(1, K + 1):
#    x_names.append('s'+ str(k))
#    x_names.append('c'+ str(k))

x_names.append('data')    

for k in range(1, Khist + 1):
    x_names.append('data_history_' + str(k))
    
#for k in range(1, Kday + 1):
#    x_names.append('data_history_day_' + str(k))


test_from = pd.to_datetime('2016-04-30 23:00:00')
test_to = pd.to_datetime('2016-05-31 17:00:00')

alphas = {1: np.arange(29, 32, 1), 2: np.arange(44, 48, 1), 3: np.arange(48, 63, 1), 
          4: np.arange(63, 67, 1), 5: np.arange(92, 97, 1), 6:  np.arange(80, 89, 1)}

topModels = {}
topAlphas = {}        
for shift in range(1, nModels + 1):
    bestModel = None
    bestAlpha = 0
    bestMae = sys.maxint

    for alpha in alphas[shift]:

        print 'shift', shift, 'alpha is ', alpha, 
        
        m = sklearn.linear_model.Lasso(alpha=alpha)

        trainF = allData[train_from : train_to]
        fitted = m.fit(trainF[x_names], trainF['target_' + str(shift)])

        mae = metrics.mean_absolute_error(allData[test_from : test_to]['target_' + str(shift)],
                                          fitted.predict(allData[test_from : test_to][x_names]))
        
        if mae < bestMae:
            bestMae = mae
            bestModel = fitted
            bestAlpha = alpha
        
        print 'MAE',  mae

    topModels[shift] = bestModel
    topAlphas[shift] = bestAlpha
    print 'shift', shift, 'best alpha is ', bestAlpha, 'MAE', bestMae


 shift 1 alpha is  29 MAE 21.08019712966646
shift 1 alpha is  30 MAE 21.080104082555252
shift 1 alpha is  31 MAE 21.080109513699828
shift 1 best alpha is  30 MAE 21.080104082555252
shift 2 alpha is  44 MAE 28.79123568906081
shift 2 alpha is  45 MAE 28.790849169432036
shift 2 alpha is  46 MAE 28.79064545065798
shift 2 alpha is  47 MAE 28.790915733855737
shift 2 best alpha is  46 MAE 28.79064545065798
shift 3 alpha is  48 MAE 32.53717584992565
shift 3 alpha is  49 MAE 32.53693364494074
shift 3 alpha is  50 MAE 32.536723777913856
shift 3 alpha is  51 MAE 32.53661211594691
shift 3 alpha is  52 MAE 32.53658299967899
shift 3 alpha is  53 MAE 32.53664217211369
shift 3 alpha is  54 MAE 32.53673163367521
shift 3 alpha is  55 MAE 32.53691226976218
shift 3 alpha is  56 MAE 32.53721703127389
shift 3 alpha is  57 MAE 32.537613338791324
shift 3 alpha is  58 MAE 32.53814897865414
shift 3 alpha is  59 MAE 32.53868826970786
shift 3 alpha is  60 MAE 32.539338007778746
shift 3 alpha is  61 MAE 32.5400583

In [39]:
topAlphas

{1: 30, 2: 46, 3: 52, 4: 64, 5: 93, 6: 83}

Посчитаем метрику за май итеративно по каждому региону

In [43]:
test_from = pd.to_datetime('2016-04-30 23:00:00')


totalMay = 0
regMaes = []
for reg in data.region.unique():
    regMae = 0  
    regData = allData[allData['region'] == reg]
    
    for ix in range(0, 739):
        tPred = test_from + datetime.timedelta(hours=ix)
        
        maes = []
        for shift in range(1, nModels + 1):
            mae = metrics.mean_absolute_error(regData[tPred:tPred]['target_' + str(shift)], 
                                              topModels[shift].predict(regData[tPred:tPred][x_names]))
            maes.append(mae)
        iterMae = np.sum(maes)
        #print 'test mae is ', np.mean(maes)
        regMae += iterMae
    totalMay += regMae 
    regMaes.append(regMae)
    clear_output(wait=True)
    print 'region ', reg, 'region MAE', regMae, 'total MAE', totalMay

region  1684 region MAE 37745.257078123 total MAE 14125936.887141982


In [44]:
Qmay = totalMay / (data.region.unique().shape[0] * 739 * 6)
print Qmay

31.233553749418448


Предыдущее значение метрики за май было 33.04337465998621. Есть некоторое улучшение.

Посчитаем метрику за июнь итеративно по каждому региону

In [55]:
test_from = pd.to_datetime('2016-05-31 23:00:00')

juneResults = []

totalJune = 0
regMaes = []
for reg in data.region.unique():
    regMae = 0  
    regData = allData[allData['region'] == reg]
    
    for ix in range(0, 715):
        tPred = test_from + datetime.timedelta(hours=ix)
        
        maes = []
        for shift in range(1, nModels + 1):
            pred = topModels[shift].predict(regData[tPred:tPred][x_names])
            
            pdId = str.format("{0}_{1}_{2}_{3}",reg,tPred.date(), tPred.hour, shift)
            juneResults.append((pdId, pred[0]))
            
            mae = metrics.mean_absolute_error(regData[tPred:tPred]['target_' + str(shift)], pred)
            maes.append(mae)
        iterMae = np.sum(maes)
        #print 'test mae is ', np.mean(maes)
        regMae += iterMae
    totalJune += regMae 
    regMaes.append(regMae)
    clear_output(wait=True)
    print 'region ', reg, 'region MAE', regMae, 'total MAE', totalMay

region  1684 region MAE 37501.62292749149 total MAE 14125936.887141982


In [56]:
Qjune = totalJune / (data.region.unique().shape[0] * 715 * 6)
print Qjune

29.45654190596134


Предыдущее значение метрики за июнь было 31.50576641207743. Есть некоторое улучшение.

In [57]:
print len(juneResults)

437580


In [59]:
juneDf = pd.DataFrame(juneResults)
juneDf.to_csv('TaxiData/w5_kaggle.csv',index=False)

Ссылка на сабмишен https://www.kaggle.com/submissions/7474762/7474762.zip